## Preprocessing

In [141]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [142]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'], axis = 1, inplace = True) 

In [143]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [144]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [145]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts=application_df.value_counts("APPLICATION_TYPE")
app_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [146]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = app_type_counts[app_type_counts<500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [147]:
# Look at CLASSIFICATION value counts for binning
class_value_counts = application_df.value_counts("CLASSIFICATION")
class_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [148]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_value_counts_1 = application_df.value_counts("CLASSIFICATION")
class_value_counts_1 = class_value_counts[class_value_counts>1]
class_value_counts_1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1267        2
C1246        2
C1256        2
C3200        2
C1234        2
dtype: int64

In [149]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = class_value_counts[class_value_counts<1800].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [150]:
# Look at USE_CASE value counts for binning
use_counts=application_df.value_counts("USE_CASE")
use_counts

USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
dtype: int64

In [151]:
# Look at AFFILIATION value counts for binning
aff_counts=application_df.value_counts("AFFILIATION")
aff_counts

AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
dtype: int64

In [152]:
# Look at ORGANIZATION value counts for binning
org_counts=application_df.value_counts("ORGANIZATION")
org_counts

ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
dtype: int64

In [153]:
# Choose a cutoff value and create a list of organization types to be replaced

org_types_to_replace = org_counts[org_counts<10000].index.tolist()

# Replace in dataframe
for app in org_types_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [154]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']

X = application_df.drop(columns='IS_SUCCESSFUL')



In [155]:
# X.head()

y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [156]:
# Convert categorical data to numeric with `pd.get_dummies`
X = pd.get_dummies(X)

In [157]:
X.head()

,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,0,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [158]:
 # Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [159]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [173]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=39))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_111 (Dense)           (None, 100)               4000      
                                                                 
 dense_112 (Dense)           (None, 80)                8080      
                                                                 
 dense_113 (Dense)           (None, 50)                4050      
                                                                 
 dense_114 (Dense)           (None, 20)                1020      
                                                                 
 dense_115 (Dense)           (None, 1)                 21        
                                                                 
Total params: 17,171
Trainable params: 17,171
Non-trainable params: 0
_________________________________________________________________


In [174]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5678 - accuracy: 0.7239
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7299
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7308
Epoch 4/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5503 - accuracy: 0.7317
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7329
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7332
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7347
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7346
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7352
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5452 - accura

In [175]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 63s - loss: 0.6000 - accuracy: 0.7286 - 63s/epoch - 236ms/step
Loss: 0.5999854803085327, Accuracy: 0.7286297082901001


In [176]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')